In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# Import Pandas
import pandas as pd

# PySpark imports
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.functions import count


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-02 02:43:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.7’

postgresql-42.2.16. 100%[===================>] 979.38K  5.64MB/s    in 0.2s    

2021-07-02 02:43:43 (5.64 MB/s) - ‘postgresql-42.2.16.jar.7’ saved [1002883/1002883]



In [3]:
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

### Recreate the vine_table

In [5]:
# Read in the Review dataset as a DataFrame

df.show(10)
df.printSchema()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [6]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [7]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

vine_df_20_total_votes = vine_df.filter("total_votes>=20")
vine_df_20_total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [8]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

vine_df_helpful_votes = vine_df_20_total_votes.withColumn("helpful_div_by_total", col("helpful_votes") / col("total_votes"))
vine_df_helpful_votes = vine_df_helpful_votes.filter("helpful_div_by_total>=0.5")
vine_df_helpful_votes = vine_df_helpful_votes.drop("helpful_div_by_total")
vine_df_helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|                Y|
|R230W87COFSTCL|          5|           21|         21|   N|     

In [9]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'

vine_reviews = vine_df_helpful_votes
vine_reviews = vine_reviews.filter(col("vine").contains("Y"))
vine_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|          139|        147|   Y|                N|
|R1XK3ALB45D7N4|          5|           33|         34|   Y|                N|
|R1IZCSTLX81D6C|          5|           31|         33|   Y|                N|
|R2C8NC8EQLH4JF|          3|           45|         48|   Y|                N|
|R1JJ1YOJMOML1P|          5|           18|         21|   Y|                N|
|R3TKG664L9MTXJ|          4|          164|        175|   Y|                N|
|R2E942L5EX73FP|          5|           29|         33|   Y|                N|
|R3CGRT9HYB3LI7|          5|           20|         21|   Y|                N|
|R3HIFO4D3ZIBZ7|          5|           25|         26|   Y|                N|
|R2VUXJT91MXOQJ|          3|           72|         79|   Y|     

In [10]:
# 4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'

not_vine_reviews = vine_df_helpful_votes
not_vine_reviews = not_vine_reviews.filter(col("vine").contains("N"))
not_vine_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|                Y|
|R230W87COFSTCL|          5|           21|         21|   N|     

In [11]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

vine_review_results = vine_reviews.groupby("star_rating").agg(count("star_rating"))
vine_review_results = vine_review_results.sort("star_rating", ascending=False).withColumnRenamed("count(star_rating)", "vine_star_rating_counts")
vine_review_results.show()

not_vine_review_results = not_vine_reviews.groupby("star_rating").agg(count("star_rating"))
not_vine_review_results = not_vine_review_results.sort("star_rating", ascending=False).withColumnRenamed("count(star_rating)", "not_vine_star_rating_counts")
not_vine_review_results.show()

+-----------+-----------------------+
|star_rating|vine_star_rating_counts|
+-----------+-----------------------+
|          5|                     15|
|          4|                      7|
|          3|                     10|
|          2|                      1|
+-----------+-----------------------+

+-----------+---------------------------+
|star_rating|not_vine_star_rating_counts|
+-----------+---------------------------+
|          5|                      23733|
|          4|                       7803|
|          3|                       4388|
|          2|                       3085|
|          1|                       6379|
+-----------+---------------------------+



In [12]:
# 5. con't (totals)

vine_review_results_pd = vine_review_results.toPandas()
vine_review_results_pd = vine_review_results_pd.set_index("star_rating")

vine_review_results_pd

,vine_star_rating_counts
star_rating,
5,15
4,7
3,10
2,1


In [13]:
# 5. con't (totals)

not_vine_review_results_pd = not_vine_review_results.toPandas()
not_vine_review_results_pd = not_vine_review_results_pd.set_index("star_rating")

not_vine_review_results_pd

,not_vine_star_rating_counts
star_rating,
5,23733
4,7803
3,4388
2,3085
1,6379


In [14]:
# 5. con't (totals)

vine_star_rating_total = pd.Series(data=vine_review_results_pd["vine_star_rating_counts"])
vine_star_rating_total = vine_star_rating_total.sum()
vine_star_rating_total

33

In [15]:
# 5. con't (totals)

not_vine_star_rating_total = pd.Series(data=not_vine_review_results_pd["not_vine_star_rating_counts"])
not_vine_star_rating_total = not_vine_star_rating_total.sum()
not_vine_star_rating_total

45388

In [16]:
# 5. con't (totals)

vine_review_results_pd['rating_total'] = vine_star_rating_total
vine_review_results_pd

,vine_star_rating_counts,rating_total
star_rating,,
5,15,33
4,7,33
3,10,33
2,1,33


In [17]:
# 5. con't (totals)

not_vine_review_results_pd['rating_total'] = not_vine_star_rating_total
not_vine_review_results_pd

,not_vine_star_rating_counts,rating_total
star_rating,,
5,23733,45388
4,7803,45388
3,4388,45388
2,3085,45388
1,6379,45388


In [18]:
# 5. con't (totals)

vine_review_results_pd['perc_of_rating_total'] = (vine_review_results_pd.vine_star_rating_counts / vine_review_results_pd.rating_total)
vine_review_results_pd['perc_of_rating_total'] = vine_review_results_pd['perc_of_rating_total'].map("{:.0%}".format)

vine_review_results_pd

,vine_star_rating_counts,rating_total,perc_of_rating_total
star_rating,,,
5,15,33,45%
4,7,33,21%
3,10,33,30%
2,1,33,3%


In [19]:
# 5. con't (totals)

not_vine_review_results_pd['perc_of_rating_total'] = (not_vine_review_results_pd.not_vine_star_rating_counts / not_vine_review_results_pd.rating_total)
not_vine_review_results_pd['perc_of_rating_total'] = not_vine_review_results_pd['perc_of_rating_total'].map("{:.0%}".format)

not_vine_review_results_pd

,not_vine_star_rating_counts,rating_total,perc_of_rating_total
star_rating,,,
5,23733,45388,52%
4,7803,45388,17%
3,4388,45388,10%
2,3085,45388,7%
1,6379,45388,14%


In [24]:
# 5. con't (totals)

vine_review_results_pd = vine_review_results_pd.filter(like = '5', axis=0)
vine_review_results_pd

,vine_star_rating_counts,rating_total,perc_of_rating_total
star_rating,,,
5,15,33,45%


In [25]:
# 5. con't (totals)

not_vine_review_results_pd = not_vine_review_results_pd.filter(like = '5', axis=0)
not_vine_review_results_pd

,not_vine_star_rating_counts,rating_total,perc_of_rating_total
star_rating,,,
5,23733,45388,52%


In [68]:
totals_merged = pd.merge(vine_review_results_pd, not_vine_review_results_pd, how="outer", on=["star_rating"])
totals_merged = totals_merged.rename(columns={"vine_star_rating_counts": "Vine Ct"})
totals_merged = totals_merged.rename(columns={"rating_total_x": "Vine Total"})
totals_merged = totals_merged.rename(columns={"perc_of_rating_total_x": "Vine Ct % of Total"})
totals_merged = totals_merged.rename(columns={"not_vine_star_rating_counts": "Non-Vine Ct)"})
totals_merged = totals_merged.rename(columns={"rating_total_y": "Non-Vine Total"})
totals_merged = totals_merged.rename(columns={"perc_of_rating_total_y": "Non-Vine Ct % of Total"})

totals_merged

,Vine Ct,Vine Total,Vine Ct % of Total,Non-Vine Ct),Non-Vine Total,Non-Vine Ct % of Total
star_rating,,,,,,
5,15,33,45%,23733,45388,52%


In [69]:
print("Five-Star Ratings")
totals_table = spark.createDataFrame(totals_merged)
totals_table.show()

Five-Star Ratings
+-------+----------+------------------+------------+--------------+----------------------+
|Vine Ct|Vine Total|Vine Ct % of Total|Non-Vine Ct)|Non-Vine Total|Non-Vine Ct % of Total|
+-------+----------+------------------+------------+--------------+----------------------+
|     15|        33|               45%|       23733|         45388|                   52%|
+-------+----------+------------------+------------+--------------+----------------------+

